## This Notebook shows how a raw Dataset gets processed for further machine learning steps

In [1]:
# Imports
import os, sys
import numpy as np
from typing import Tuple, List, Dict
import pandas as pd
import io

# to enable local imports
module_path = os.path.abspath('code')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
    
from machine_learning_data_generation import loadPickeldDataset, processRawDatasetToPickleFiles, loadOnlineEEGdata
from consts import DEVICES_NEUROSCAN

D:\Masterthesis\thesis_eeg\code


Using TensorFlow backend.


## Save a dataset to pickel files
With this function  a dataset gets processed and the output will be saved to pickel files

In [2]:
PROCESS_DATA = False

if PROCESS_DATA:
    # Process the online EEG Data
    processRawDatasetToPickleFiles(datasetDirPath = "D:/Masterthesis/EEG_Data/eeg_data_online",
                              device = DEVICES_NEUROSCAN,
                              awakeFileName = None,
                              fatigueFileName = "Fatigue_state_256hz.csv",
                              normalFileName = "Normal_state_256hz.csv",
                              unlabeledFileName = None)
else:
    print ("Already processed the EEG Online Data")

Already processed the EEG Online Data


### Function to create a Machine Learning Dataset
With this functions you can create a X and y Dataset from a given EEG Dataset (use the function 'processRawDatasetToPickleFiles').
It creates a X & y for the EEG Signals and the frequency features

In [3]:
from machine_learning_data_generation import createAndSafeMlDataset
from consts import TARGET_FATIGUE, TARGET_NORMAL

CREATE_ML_DATA = False

TARGET_LABEL_DICT = {TARGET_NORMAL : 0,
                     TARGET_FATIGUE : 1}

if CREATE_ML_DATA:
    
    eegDataset = loadPickeldDataset("D:/Masterthesis/EEG_Data/eeg_data_online")
    
    createAndSafeMlDataset(eegDataset=eegDataset,
                           targetLabelDict=TARGET_LABEL_DICT,
                           dirPath="D:/Masterthesis/EEG_Data/eeg_data_online")
else:
    print("Already created ML Data")


Already created ML Data


### Example - Load  the online EEG Dataset

In [4]:
eegData, freqData = loadOnlineEEGdata(shuffle=False)

Loading Online EEG Data from D:/Masterthesis/EEG_Data/eeg_data_online ...
EEG Data Shape:
(5024, 512, 40) (5024,) (2154, 512, 40) (2154,)
Freq Data Shape:
(1008, 1, 1200) (1008,) (432, 1, 1200) (432,)


In [5]:
print("Shape train X: {}".format(trainX.shape))
print("Shape train y: {}".format(trainy.shape))
print("Shape test X: {}".format(testX.shape))
print("Shape test y: {}".format(testy.shape))


NameError: name 'trainX' is not defined

array([[[ 0.58857225, -0.05947701,  0.75340673, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.77181281,  0.53140419,  0.83882233, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.60697763,  0.44422637,  0.88404084, ...,  0.        ,
          0.        ,  0.        ]],

       ...,

       [[ 0.73472731,  0.54010546,  0.80864854, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.50687363,  0.31490547,  0.60680156, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.53433675,  0.39765327,  0.74250543, ...,  0.        ,
          0.        ,  0.        ]]])

In [28]:
repeats = 5
# repeat experiment
scores = list()
for r in range(repeats):
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
# summarize results
summarize_results(scores)

>#1: 51.620
>#2: 51.620
>#3: 51.620
>#4: 51.620
>#5: 51.620
[51.62037014961243, 51.62037014961243, 51.62037014961243, 51.62037014961243, 51.62037014961243]
Accuracy: 51.620% (+/-0.000)
